In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np
import config
import tensorflow as tf
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm
from merlin.models.tf.core.aggregation import SequenceAggregator


dmodel = 64
BATCH_SIZE = 1024
LEARNING_RATE = 0.005
TOP = 10

2023-09-10 17:09:41.463343: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-10 17:09:41.846520: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
train = Dataset(os.path.join(config.data_final,'transformer4rec', 'train.parquet/'),engine="parquet",  part_size="1GB")
valid = Dataset(os.path.join(config.data_final,'transformer4rec', 'valid.parquet/'),engine="parquet", part_size="1GB")
print(len(train.to_ddf()), len(valid.to_ddf()))

KeyError: 'article_id_count_norm'

In [3]:
target_schema = train.schema.select_by_tag(Tags.ITEM_ID).column_names[0]


train.schema = train.schema.select_by_tag(Tags.SEQUENCE).select_by_tag(Tags.CATEGORICAL)
valid.schema = train.schema


In [4]:
target = train.schema.select_by_tag(Tags.ITEM_ID).column_names[0]
target

'article_id_list'

In [6]:
input_block = mm.InputBlockV2(
    train.schema,    
    embeddings=mm.Embeddings(
        train.schema, 
        sequence_combiner=None,
        dim=dmodel
        ),
)

In [7]:
batch = mm.sample_batch(train, batch_size=BATCH_SIZE, include_targets=False)
print(input_block(batch).shape)

(1024, None, 1024)


In [8]:
input_block(batch).shape

TensorShape([1024, None, 1024])

In [9]:
mlp_block = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )

In [10]:
mlp_block(input_block(batch)).shape

2023-09-09 15:40:50.460524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


TensorShape([1024, None, 64])

In [11]:
xlnet_block = mm.XLNetBlock(d_model=dmodel, n_head=4, n_layer=2)

In [12]:
dense_block = mm.SequentialBlock(
    input_block,
    mlp_block,
    xlnet_block
)

In [13]:
item_id_name = train.schema.select_by_tag(Tags.ITEM_ID).first.properties['domain']['name']
print(item_id_name)

article_id


In [14]:
prediction_task = mm.CategoricalOutput(
    to_call=input_block["categorical"][item_id_name],
)

In [15]:
mlp_block2 = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )

In [16]:
model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

In [17]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
)

In [23]:
model_transformer

Model(
  (_should_compute_train_metrics_for_batch): <tf.Variable 'should_compute_train_metrics_for_batch:0' shape=() dtype=bool, numpy=True>
  (blocks): _TupleWrapper((SequentialBlock(
    (layers): List(
      (0): ParallelBlock(
        (_aggregation): ConcatFeatures(
          (_feature_shapes): Dict(
            (article_id_list): TensorShape([512, None, 1])
            (day_list): TensorShape([512, None, 1])
            (week_list): TensorShape([512, None, 1])
            (month_list): TensorShape([512, None, 1])
            (year_list): TensorShape([512, None, 1])
            (product_code_list): TensorShape([512, None, 1])
            (product_type_no_list): TensorShape([512, None, 1])
            (graphical_appearance_no_list): TensorShape([512, None, 1])
            (colour_group_code_list): TensorShape([512, None, 1])
            (perceived_colour_value_id_list): TensorShape([512, None, 1])
            (perceived_colour_master_id_list): TensorShape([512, None, 1])
           

In [22]:

model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
              metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[5])
             )
model_transformer.fit(train,  batch_size=512, epochs=30, pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block))

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/30


2023-09-09 15:43:01.034320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-09-09 15:43:01.203034: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2023-09-09 15:43:11.703972: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fef2e080970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-09 15:43:11.703999: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-09-09 15:43:11.706703: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-09 15:43:11.774628: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 210/1354 [===>..........................] - ETA: 4:59 - loss: 9.1503 - recall_at_5: 0.0067 - mrr_at_5: 0.0033 - ndcg_at_5: 0.0041 - map_at_5: 0.0033 - precision_at_5: 0.0013 - regularization_loss: 0.0000e+00 - loss_batch: 9.1491

KeyboardInterrupt: 

In [21]:
model_transformer.evaluate(
    valid,
    batch_size=1024,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block),
    return_dict=True,
)

ValueError: To evaluate using `SequencePredictNext`, ensure that your TransformerBlock has `masking_pre` set as `combinators.SequentialBlock(    [SequenceCausalLastInference(), ExtractMaskFromTargets()])`. You can automatically set `masking_pre` by passing `SequencePredictNext` as the `pre` argument to the `fit` method: `model.fit(..., pre=SequencePredictNext(...))`.

In [ ]:
train.to_ddf().compute()

KeyError: 'Column price_min does not exist in schema'